Algorithm:
    
Step 1: Read the image using opencv
    
Step 2: Pass the image to Yolo Network for detection
    
Step 3: Trained Objects are detected [x,y,w,h] format     

# Prepare Custom Dataset

Procedure:
    
1. Dataset is annotated using "LabelImg" tool in "YOLO" format in python [reference: https://pypi.org/project/labelImg/1.4.0/]

2. "data" folder is created to training. [It contains "obj" folder, "obj.names", "obj.data", "train.txt", "val.txt" and "yolov3-tiny-obj.cfg" files]

3. In "yolov3-tiny-obj.cfg", We need to change the no of classes that we have annotated and also some hyper parameter tuning regarding our custom dataset.

4. After preparing "data" folder training is done using through darknet.[Reference:"Yolo_custom_Training.ipynb"]   

To generate "train.txt or val.txt"

In [ ]:
import os
path = ".\data\obj"
img_name = []
for filename in (os.listdir(path)):
    Image_file = filename #dumy file name
    file = filename.split('.') #code to split the filename to check it is json r not
    if file[1] == 'png' or file[1] == 'jpeg' or file[1] == 'jpg':
        rename = file[0] + ".jpg"
        os.rename(path + '/' + filename, path + '/' + rename)
        if os.path.exists(path + '/' + file[0] + '.txt') and os.path.exists(path + '/' + file[0] + '.jpg'):
            txt_out = ("data/obj/" + Image_file)
            f = open(r"./data/train.txt", "a+")
            f.write(txt_out)
            f.write("\n")
            f.close()
print("\n!!! Train.TXT Generated !!!\n")

In [7]:
#dependcies
import cv2
import numpy as np
import pandas as pd
import glob

In [3]:
def load_classes(classes):
    '''Used to load classes in list format'''
    with open(classes, 'r') as f:
        class_names = f.read().splitlines()
    return class_names

def get_output_format(box):
    x, y, w, h = box
    return int(x), int(y), int(x + w), int(y + h)

def bounding_box(img, conf=0.2, nms_thresh=0.3):
    '''This function detects the bounding box of given image'''
    global net, classes, outputlayers
    
    result = {k: [] for k in classes}
    confidences = {k: [] for k in classes}
    boxes = {k: [] for k in classes}
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(outputlayers)
    Height, Width, _ = img.shape
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - (w / 2)
                y = center_y - (h / 2)
                confidences[classes[class_id]].append(float(confidence))
                boxes[classes[class_id]].append([int(i) for i in [x, y, w, h]])
    indices = {}
    for class_name, box in boxes.items():
        indices[class_name] = cv2.dnn.NMSBoxes(box, confidences[class_name], conf,nms_thresh)
    
    for key, index in indices.items():
        for i in index:
            try:
                select = i[0]
            except:  
                select = i
            x, y, w, h = boxes[key][select]
#             x1,y1,x2,y2=int(x), int(y), int(x + w), int(y + h)
            result[key].append((x, y, w, h))

    return result


# Inference

In [4]:
def data_format_pd(res_dic,img_name,classes):
    data=[]
    for cls, det in res_dic.items():
        for box in det:
            data.append([img_name,cls,classes.index(cls),list(box)])
    return data    

    

In [13]:
model_cfg_path="./Yolo/yolo_v3.cfg"
model_weights_path="./Yolo/yolov3-tiny.weights"
classes_txt_path="./Yolo/classes.txt"

net = cv2.dnn.readNetFromDarknet(model_cfg_path, model_weights_path)
classes = load_classes(classes_txt_path)
layer_names = net.getLayerNames()
try:
    outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
except:
    outputlayers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

data=[]    
for each_img in glob.glob('./Object_detection_TestSet/*.jpg'):
    file=each_img.split('\\')[-1]
    img=cv2.imread(each_img)
    res=bounding_box(img)
    data=data+data_format_pd(res,file,classes)
    

# Saving in excel

In [14]:
df=pd.DataFrame(data,columns=['file_name','class','unique_id','coordinates'])
print(df.head())

df.to_excel("sample.xlsx") #dataframe to excel

   file_name              class  unique_id            coordinates
0  Img10.jpg  Standard Vehicles          0   [1129, 264, 99, 121]
1  Img10.jpg  Standard Vehicles          0     [753, 208, 55, 73]
2  Img10.jpg  Standard Vehicles          0  [1459, 343, 123, 134]
3  Img10.jpg       Motorcyclist          2     [864, 222, 33, 79]
4  Img10.jpg         Pedestrian          3    [381, 120, 46, 242]
